In [1]:
import trimesh

scene = trimesh.Scene()

In [6]:
import torch
import numpy as np
import os
import shutil
from tqdm import tqdm
import yaml

import sys
import os

# models
from my_code.models.diag_conditional import DiagConditionedUnet
from diffusers import DDPMScheduler

import my_code.datasets.template_dataset as template_dataset

import my_code.diffusion_training_sign_corr.data_loading as data_loading

import networks.diffusion_network as diffusion_network
import matplotlib.pyplot as plt
import my_code.utils.plotting_utils as plotting_utils
import utils.fmap_util as fmap_util
import metrics.geodist_metric as geodist_metric
from my_code.sign_canonicalization.training import predict_sign_change
import argparse
from pyFM_fork.pyFM.refine.zoomout import zoomout_refine
import my_code.utils.zoomout_custom as zoomout_custom
from utils.shape_util import compute_geodesic_distmat
from my_code.diffusion_training_sign_corr.test.test_diffusion_cond import select_p2p_map_dirichlet, log_to_database, parse_args
import accelerate
import my_code.sign_canonicalization.test_sign_correction as test_sign_correction
import networks.fmap_network as fmap_network
from my_code.utils.median_p2p_map import dirichlet_energy

import torch
import numpy as np
import os
import shutil
from tqdm import tqdm
import yaml

import sys
import os

# models
from my_code.models.diag_conditional import DiagConditionedUnet
from diffusers import DDPMScheduler

import my_code.datasets.template_dataset as template_dataset

import my_code.diffusion_training_sign_corr.data_loading as data_loading

import networks.diffusion_network as diffusion_network
import matplotlib.pyplot as plt
import my_code.utils.plotting_utils as plotting_utils
import utils.fmap_util as fmap_util
import metrics.geodist_metric as geodist_metric
from my_code.sign_canonicalization.training import predict_sign_change
import argparse
from pyFM_fork.pyFM.refine.zoomout import zoomout_refine
import my_code.utils.zoomout_custom as zoomout_custom
from utils.shape_util import compute_geodesic_distmat
from my_code.diffusion_training_sign_corr.test.test_diffusion_cond import select_p2p_map_dirichlet, log_to_database, parse_args
import accelerate
import my_code.sign_canonicalization.test_sign_correction as test_sign_correction
import networks.fmap_network as fmap_network
from my_code.utils.median_p2p_map import dirichlet_energy

tqdm._instances.clear()

class RegularizedFMNet(torch.nn.Module):
    """Compute the functional map matrix representation in DPFM"""
    def __init__(self, lmbda=0.01, resolvant_gamma=0.5, bidirectional=False):
        super(RegularizedFMNet, self).__init__()
        self.lmbda = lmbda
        self.resolvant_gamma = resolvant_gamma
        self.bidirectional = bidirectional

    def compute_functional_map(self, A, B, evals_x, evals_y):
        # A = torch.bmm(evecs_trans_x, feat_x)  # [B, K, C]
        # B = torch.bmm(evecs_trans_y, feat_y)  # [B, K, C]

        D = fmap_network.get_mask(evals_x, evals_y, self.resolvant_gamma)  # [B, K, K]

        A_t = A.transpose(1, 2)  # [B, C, K]
        A_A_t = torch.bmm(A, A_t)  # [B, K, K]
        B_A_t = torch.bmm(B, A_t)  # [B, K, K]

        C_i = []
        for i in range(evals_x.shape[1]):
            D_i = torch.cat([torch.diag(D[bs, i, :].flatten()).unsqueeze(0) for bs in range(evals_x.shape[0])], dim=0)
            C = torch.bmm(torch.inverse(A_A_t + self.lmbda * D_i), B_A_t[:, [i], :].transpose(1, 2))
            C_i.append(C.transpose(1, 2))

        Cxy = torch.cat(C_i, dim=1)
         
        return Cxy
    



def get_geo_error(
    p2p_first, p2p_second,
    evecs_first, evecs_second,
    corr_first, corr_second,
    num_evecs, apply_zoomout,
    dist_x,
    regularized=False,
    evecs_trans_first=None, evecs_trans_second=None,
    evals_first=None, evals_second=None,
    return_p2p=False, return_Cxy=False,
    A2=None, fmnet=None
    ):
        
    if regularized:
        Cxy = fmnet.compute_functional_map(
            evecs_trans_second[:num_evecs, p2p_second].unsqueeze(0),
            evecs_trans_first[:num_evecs, p2p_first].unsqueeze(0),
            evals_second[:num_evecs].unsqueeze(0),
            evals_first[:num_evecs].unsqueeze(0), 
        )[0].T
        
    else:
        Cxy = torch.linalg.lstsq(
            evecs_second[:, :num_evecs][p2p_second],
            evecs_first[:, :num_evecs][p2p_first]
            ).solution
    
    
    if apply_zoomout:
        Cxy = zoomout_custom.zoomout(
            FM_12=Cxy, 
            evects1=evecs_first,
            evects2=evecs_second,
            nit=evecs_first.shape[1] - num_evecs, step=1,
            A2=A2
        )
        num_evecs = evecs_first.shape[1]
        
    p2p = fmap_util.fmap2pointmap(
        C12=Cxy,
        evecs_x=evecs_first[:, :num_evecs],
        evecs_y=evecs_second[:, :num_evecs],
        ).cpu()
    
    geo_err = geodist_metric.calculate_geodesic_error(
        dist_x, corr_first.cpu(), corr_second.cpu(), p2p, return_mean=True
    )
    
    # if return_p2p:
    #     return geo_err * 100, p2p
    # else:
    #     return geo_err * 100
    
    if not return_p2p and not return_Cxy:
        return geo_err * 100
    
    payload = [geo_err * 100]
    
    if return_p2p:
        payload.append(p2p)
    if return_Cxy:
        payload.append(Cxy)
        
    return payload


def filter_p2p_by_confidence(
        p2p_first, p2p_second,
        confidence_scores_first, confidence_scores_second,
        confidence_threshold, log_file_name
    ):
    
    assert p2p_first.shape[0] == p2p_second.shape[0]
    
    # select points with both confidence scores above threshold
    valid_points = (confidence_scores_first < confidence_threshold) & (confidence_scores_second < confidence_threshold)
    
    with open(log_file_name, 'a') as f:
        
        while valid_points.sum() < 0.05 * len(valid_points):
            confidence_threshold += 0.05
            valid_points = (confidence_scores_first < confidence_threshold) & (confidence_scores_second < confidence_threshold)
            
            f.write(f'Increasing confidence threshold: {confidence_threshold}\n')
        f.write(f'Valid points: {valid_points.sum()}\n')
        assert valid_points.sum() > 0, "No valid points found"
        
    p2p_first = p2p_first[valid_points]
    p2p_second = p2p_second[valid_points]
    
    return p2p_first, p2p_second

In [9]:
def get_fmaps_evec_signs(
        data, model,
        noise_scheduler, config, args,
        template_shape, sign_corr_net,
        return_fmaps
    ):
        
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    num_evecs = config["model_params"]["sample_size"]

        
    verts_first = template_shape['verts'].unsqueeze(0).to(device)
    verts_second = data['verts'].unsqueeze(0).to(device)
    
    faces_first = template_shape['faces'].unsqueeze(0).to(device)
    faces_second = data['faces'].unsqueeze(0).to(device)

    evecs_first = template_shape['evecs'][:, :num_evecs].unsqueeze(0).to(device)
    evecs_second = data['evecs'][:, :num_evecs].unsqueeze(0).to(device)
    

    if config["sign_net"]["with_mass"]:
        mass_mat_first = torch.diag_embed(
            template_shape['mass'].unsqueeze(0)
            ).to(device)
        mass_mat_second = torch.diag_embed(
            data['mass'].unsqueeze(0)
            ).to(device)
    else:
        mass_mat_first = None
        mass_mat_second = None


    ###############################################
    # get conditioning and signs num_iters_avg times
    ###############################################

    evecs_cond_first_list = []
    evecs_cond_second_list = []
    evecs_first_signs_list = []
    evecs_second_signs_list = []

    for _ in range(args.num_iters_avg):

        # predict the sign change
        with torch.no_grad():
            sign_pred_first, support_vector_norm_first, _ = predict_sign_change(
                sign_corr_net, verts_first, faces_first, evecs_first, 
                mass_mat=mass_mat_first, input_type=sign_corr_net.input_type,
                evecs_per_support=config["sign_net"]["evecs_per_support"],
                
                mass=template_shape['mass'].unsqueeze(0), L=template_shape['L'].unsqueeze(0),
                evals=template_shape['evals'][:config["sign_net"]["net_params"]["k_eig"]].unsqueeze(0),
                evecs=template_shape['evecs'][:,:config["sign_net"]["net_params"]["k_eig"]].unsqueeze(0),
                gradX=template_shape['gradX'].unsqueeze(0), gradY=template_shape['gradY'].unsqueeze(0)
                )
            sign_pred_second, support_vector_norm_second, _ = predict_sign_change(
                sign_corr_net, verts_second, faces_second, evecs_second, 
                mass_mat=mass_mat_second, input_type=sign_corr_net.input_type,
                evecs_per_support=config["sign_net"]["evecs_per_support"],
                
                mass=data['mass'].unsqueeze(0), L=data['L'].unsqueeze(0),
                evals=data['evals'][:config["sign_net"]["net_params"]["k_eig"]].unsqueeze(0),
                evecs=data['evecs'][:,:config["sign_net"]["net_params"]["k_eig"]].unsqueeze(0),
                gradX=data['gradX'].unsqueeze(0), gradY=data['gradY'].unsqueeze(0)
                )

        # correct the evecs
        evecs_first_corrected = evecs_first.cpu()[0] * torch.sign(sign_pred_first).cpu()
        # evecs_first_corrected_norm = evecs_first_corrected / torch.norm(evecs_first_corrected, dim=0, keepdim=True)
        evecs_first_corrected_norm = torch.nn.functional.normalize(evecs_first_corrected, p=2, dim=0)
        
        evecs_second_corrected = evecs_second.cpu()[0] * torch.sign(sign_pred_second).cpu()
        # evecs_second_corrected_norm = evecs_second_corrected / torch.norm(evecs_second_corrected, dim=0, keepdim=True)
        evecs_second_corrected_norm = torch.nn.functional.normalize(evecs_second_corrected, p=2, dim=0)
        

        # product with support
        if config["sign_net"]["with_mass"]:
        # if config["sign_net"]['cond_mass_normalize']:
            
            mass_mat_first = torch.diag_embed(
                template_shape['mass'].unsqueeze(0)
                ).to(device)
            mass_mat_second = torch.diag_embed(
                data['mass'].unsqueeze(0)
                ).to(device)
            
            evecs_cond_first = torch.nn.functional.normalize(
                support_vector_norm_first[0].cpu().transpose(0, 1) \
                    @ mass_mat_first[0].cpu(),
                p=2, dim=1) \
                    @ evecs_first_corrected_norm
            
            evecs_cond_second = torch.nn.functional.normalize(
                support_vector_norm_second[0].cpu().transpose(0, 1) \
                    @ mass_mat_second[0].cpu(),
                p=2, dim=1) \
                    @ evecs_second_corrected_norm 
            
        else:
            evecs_cond_first = support_vector_norm_first[0].cpu().transpose(0, 1) @ evecs_first_corrected_norm
            evecs_cond_second = support_vector_norm_second[0].cpu().transpose(0, 1) @ evecs_second_corrected_norm
            
        evecs_cond_first_list.append(evecs_cond_first)
        evecs_cond_second_list.append(evecs_cond_second)
        evecs_first_signs_list.append(torch.sign(sign_pred_first).cpu())
        evecs_second_signs_list.append(torch.sign(sign_pred_second).cpu())
        
    evecs_cond_first_list = torch.stack(evecs_cond_first_list)
    evecs_cond_second_list = torch.stack(evecs_cond_second_list)
    evecs_first_signs_list = torch.stack(evecs_first_signs_list)
    evecs_second_signs_list = torch.stack(evecs_second_signs_list)    
    
    
    ###############################################
    # Conditioning
    ###############################################

    conditioning = torch.cat(
        (evecs_cond_first_list.unsqueeze(1), evecs_cond_second_list.unsqueeze(1)),
        1)
    
    ###############################################
    # Sample the model
    ###############################################
    
    if return_fmaps:
    
        x_sampled = torch.rand(args.num_iters_avg, 1, 
                            config["model_params"]["sample_size"],
                            config["model_params"]["sample_size"]).to(device)
        y = conditioning.to(device)    
            
        # Sampling loop
        for t in noise_scheduler.timesteps:

            # Get model pred
            with torch.no_grad():
                residual = model(x_sampled, t,
                                    conditioning=y
                                    ).sample

            # Update sample with step
            x_sampled = noise_scheduler.step(residual, t, x_sampled).prev_sample
        
    else:
        x_sampled = None
        
    return x_sampled, evecs_first_signs_list, evecs_second_signs_list



In [103]:
class Arguments:
    def __init__(self):
        self.experiment_name='partial_0.8_5k_xyz_32_2_lambda_0.001_anisRemesh_cuts_bbox_partial_0.8_xy'
        self.checkpoint_name='epoch_90'
        
        self.dataset_name='FAUST_orig_pair'
        self.split='test'
        
        self.num_iters_avg=32
        self.num_samples_median=4
        self.confidence_threshold=0.3
        
        self.smoothing_type=None
        self.smoothing_iter=None
        
        self.sign_net = 'test_partial_5k_xyz_32_1'
        
        

In [104]:
args = Arguments()

# configuration
experiment_name = args.experiment_name
checkpoint_name = args.checkpoint_name

### config
exp_base_folder = f'/home/s94zalek_hpc/shape_matching/my_code/experiments/ddpm/{experiment_name}'
with open(f'{exp_base_folder}/config.yaml', 'r') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)


sign_net_path = f'/home/s94zalek_hpc/shape_matching/my_code/experiments/sign_net/{args.sign_net}'
with open(f'{sign_net_path}/config.yaml', 'r') as f:
    sign_net_config = yaml.load(f, Loader=yaml.FullLoader)
    
    config["sign_net"] = sign_net_config
    config["sign_net"]["net_path"] = sign_net_path



### model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = DiagConditionedUnet(config["model_params"])

if "accelerate" in config and config["accelerate"]:
    accelerate.load_checkpoint_in_model(model, f"{exp_base_folder}/checkpoints/{checkpoint_name}/model.safetensors")
else:
    model.load_state_dict(torch.load(f"{exp_base_folder}/checkpoints/{checkpoint_name}"))

model.to(device)

### Sign correction network
sign_corr_net = diffusion_network.DiffusionNet(
    **config["sign_net"]["net_params"]
    )        
sign_corr_net.load_state_dict(torch.load(
        f'{config["sign_net"]["net_path"]}/{config["sign_net"]["n_iter"]}.pth'
        ))
sign_corr_net.to(device)


### noise scheduler
noise_scheduler = DDPMScheduler(num_train_timesteps=1000, beta_schedule='squaredcos_cap_v2',
                                clip_sample=True) 

# fmap network
fmnet = RegularizedFMNet(lmbda=0.01, resolvant_gamma=0.5)


### test dataset
dataset_name = args.dataset_name
split = args.split

single_dataset, test_dataset = data_loading.get_val_dataset(
    dataset_name, split, 200, preload=False, return_evecs=True, centering='bbox'
    )
# sign_corr_net.cache_dir = single_dataset.lb_cache_dir

num_evecs = config["model_params"]["sample_size"]

##########################################
# Template
##########################################

# template_shape = template_dataset.get_template(
#     num_evecs=200,
#     centering='bbox',
#     template_path=f'/home/s94zalek_hpc/shape_matching/data/SURREAL_full/template/{config["sign_net"]["template_type"]}/template.off',
#     template_corr=np.loadtxt(
#         f'/home/s94zalek_hpc/shape_matching/data/SURREAL_full/template/{config["sign_net"]["template_type"]}/corr.txt',
#         dtype=np.int32) - 1
#     )    

##########################################
# Logging
##########################################

if args.smoothing_type is not None:
    test_name = f'{args.smoothing_type}-{args.smoothing_iter}'
else:
    test_name = 'no_smoothing'

log_dir = f'{exp_base_folder}/eval/{checkpoint_name}/{dataset_name}-{split}/{test_name}'
os.makedirs(log_dir, exist_ok=True)

fig_dir = f'{log_dir}/figs'
os.makedirs(fig_dir, exist_ok=True)

log_file_name = f'{log_dir}/log_{test_name}.txt'



In [105]:
from my_code.datasets.surreal_dataset_3dc import TemplateSurrealDataset3DC


augmentations = {
        "remesh": {
            "isotropic": {
                "n_remesh_iters": 10,
                "remesh_targetlen": 1,
                "simplify_strength_min": 0.2,
                "simplify_strength_max": 0.8,
            },
            "anisotropic": {
                "probability": 0.35,
                    
                "n_remesh_iters": 10,
                "fraction_to_simplify_min": 0.2,
                "fraction_to_simplify_max": 0.6,
                "simplify_strength_min": 0.2,
                "simplify_strength_max": 0.5,
                "weighted_by": "face_count",
            },
            "partial": {
                "probability": 1,
                "n_remesh_iters": 10,
                "fraction_to_keep_min": 0.5,
                "fraction_to_keep_max": 0.8,
                "n_seed_samples": [1, 5, 25],
                "weighted_by": "face_count",
            },
        },
    }

    

test_dataset = TemplateSurrealDataset3DC(
    shape_path='/lustre/mlnvme/data/s94zalek_hpc-shape_matching/mmap_datas_surreal_train.pth',
    num_evecs=128,
    cache_lb_dir=None,
    return_evecs=True,
    return_fmap=False,
    mmap=True,
    augmentations=augmentations,
    template_path=f'/home/s94zalek_hpc/shape_matching/data/SURREAL_full/template/remeshed/template.off',
    template_corr=np.loadtxt(
        f'/home/s94zalek_hpc/shape_matching/data/SURREAL_full/template/remeshed/corr.txt',
        dtype=np.int32) - 1,
    
    # template_corr=np.loadtxt(
    #     f'/home/s94zalek_hpc/shape_matching/data/SURREAL_full/template/remeshed/corr_symmetric.txt',
    #     dtype=np.int32) - 1,
    
    centering='bbox',
)   

# print('!!! Using symmetric correspondence !!!')

In [ ]:
data = test_dataset[1]        

evecs_first = data['first']['evecs'][:, :].to(device)
evecs_second = data['second']['evecs'][:, :].to(device)

evecs_trans_first = data['first']['evecs_trans'][:, :].to(device)
evecs_trans_second = data['second']['evecs_trans'][:, :].to(device)

evals_first = data['first']['evals'][:num_evecs].to(device)
evals_second = data['second']['evals'][:num_evecs].to(device)

corr_first = data['first']['corr'].to(device)
corr_second = data['second']['corr'].to(device)

mass_second = data['second']['mass'].to(device)

###############################################
# Functional maps
###############################################

# second mesh


if config["fmap_direction"] == 'xy':

    _, evecs_first_signs_list_second, evecs_second_signs_list_second = get_fmaps_evec_signs(
        data['second'], model,
        noise_scheduler, config, args,
        data['first'], sign_corr_net,
        return_fmaps=False
    )
    # transpose the functional maps
    # Cyx_second_list = Cxy_second_list.transpose(2, 3)
    
else:
    
    _, evecs_first_signs_list_second, evecs_second_signs_list_second = get_fmaps_evec_signs(
        data['second'], model,
        noise_scheduler, config, args,
        data['first'], sign_corr_net,
        return_fmaps=False
    )
    
    # Cxy_second_list = Cyx_second_list.transpose(2, 3)
 
 
print('first', get_n_equal_signs(evecs_first_signs_list_second))
print('second', get_n_equal_signs(evecs_second_signs_list_second))
      
      
import my_code.utils.plotting_utils as plotting_utils

scene.geometry.clear()

mesh_1 = trimesh.Trimesh(vertices=data['first']['verts'].cpu(), faces=data['first']['faces'].cpu())
mesh_2 = trimesh.Trimesh(vertices=data['second']['verts'].cpu() + torch.tensor([1, 0, 0]), faces=data['second']['faces'].cpu())

scene.add_geometry(mesh_1)
scene.add_geometry(mesh_2)

scene.show()     

In [90]:
def get_n_equal_signs(evecs_signs_list):

    # n_same = torch.zeros(evecs_signs_list.shape[0])
    # for i in range(evecs_signs_list.shape[0]):
    #     for j in range(evecs_signs_list.shape[0]):
    #         n_same[i] += (evecs_signs_list[i] == evecs_signs_list[j]).all().int()

    # return n_same
    
    unique_sign_combinations = torch.unique(evecs_signs_list, dim=0)
    n_same = torch.zeros(unique_sign_combinations.shape[0])
    
    for i in range(unique_sign_combinations.shape[0]):
        for j in range(evecs_signs_list.shape[0]):
            n_same[i] += (unique_sign_combinations[i] == evecs_signs_list[j]).all().int()
            
    return n_same
    


# Gradients

In [111]:
verts_first = data['first']['verts']
gradX_first = data['first']['gradX']
gradY_first = data['first']['gradY']

verts_second = data['second']['verts']
gradX_second = data['second']['gradX']
gradY_second = data['second']['gradY']

corr_first = data['first']['corr']
corr_second = data['second']['corr']

In [116]:
prodX_first = gradX_first.to_dense() @ verts_first
prodY_first = gradY_first.to_dense() @ verts_first

prodX_second = gradX_second.to_dense() @ verts_second
prodY_second = gradY_second.to_dense() @ verts_second

(prodX_first[corr_first] @ prodX_second.T).diagonal()

In [122]:
template_original = template_dataset.get_template(
    num_evecs=single_dataset.num_evecs,
    centering='bbox',
    template_path=f'/home/s94zalek_hpc/shape_matching/data/SURREAL_full/template/original/template.off',
    template_corr=np.loadtxt(
        f'/home/s94zalek_hpc/shape_matching/data/SURREAL_full/template/original/corr.txt',
        dtype=np.int32) - 1
    ) 

template_remeshed = template_dataset.get_template(
    num_evecs=single_dataset.num_evecs,
    centering='bbox',
    template_path=f'/home/s94zalek_hpc/shape_matching/data/SURREAL_full/template/remeshed/template.off',
    template_corr=np.loadtxt(
        f'/home/s94zalek_hpc/shape_matching/data/SURREAL_full/template/remeshed/corr.txt',
        dtype=np.int32) - 1
    ) 

template_symmetric = template_dataset.get_template(
    num_evecs=single_dataset.num_evecs,
    centering='bbox',
    template_path=f'/home/s94zalek_hpc/shape_matching/data/SURREAL_full/template/remeshed/template.off',
    template_corr=np.loadtxt(
        f'/home/s94zalek_hpc/shape_matching/data/SURREAL_full/template/remeshed/corr_symmetric.txt',
        dtype=np.int32) - 1
    ) 

In [140]:
def project_grad(verts_first, verts_second, gradX_first, gradY_first, gradX_second, gradY_second, corr_first, corr_second):
    prodX_first = gradX_first.to_dense() @ verts_first
    prodY_first = gradY_first.to_dense() @ verts_first

    prodX_second = gradX_second.to_dense() @ verts_second
    prodY_second = gradY_second.to_dense() @ verts_second

    print(torch.linalg.cross(prodX_first[corr_first], prodX_second[corr_second], dim=1))
    print(torch.linalg.cross(prodY_first[corr_first], prodY_second[corr_second], dim=1))
    

    # x_score = (prodX_first[corr_first] @ prodX_second[corr_second].T).diagonal().sum()
    # y_score = (prodY_first[corr_first] @ prodY_second[corr_second].T).diagonal().sum()
    
    # return torch.sqrt(x_score**2 + y_score**2)

In [ ]:
print(project_grad(
    verts_first = template_original['verts'], 
    verts_second = template_remeshed['verts'],
    gradX_first=template_original['gradX'], 
    gradY_first=template_original['gradY'], 
    gradX_second=template_remeshed['gradX'], 
    gradY_second=template_remeshed['gradY'], 
    corr_first=template_original['corr'], 
    corr_second=template_remeshed['corr']
    ))

print(project_grad(
    verts_first = template_original['verts'], 
    verts_second = template_symmetric['verts'],
    gradX_first=template_original['gradX'], 
    gradY_first=template_original['gradY'], 
    gradX_second=template_symmetric['gradX'], 
    gradY_second=template_symmetric['gradY'], 
    corr_first=template_original['corr'], 
    corr_second=template_symmetric['corr']
    ))